In [ ]:
import torch

In [4]:
# https://gist.github.com/ncullen93/58e71c4303b89e420bd8e0b0aa54bf48
def pearsonr(x, y):
    """
    Mimics `scipy.stats.pearsonr`
    Arguments
    ---------
    x : 1D torch.Tensor (N,)
    y : 1D torch.Tensor (N,)
    Returns
    -------
    r_val : float
        pearsonr correlation coefficient between x and y
    
    Scipy docs ref:
        https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html
    
    Scipy code ref:
        https://github.com/scipy/scipy/blob/v0.19.0/scipy/stats/stats.py#L2975-L3033
    Example:
        >>> x = np.random.randn(100)
        >>> y = np.random.randn(100)
        >>> sp_corr = scipy.stats.pearsonr(x, y)[0]
        >>> th_corr = pearsonr(torch.from_numpy(x), torch.from_numpy(y))
        >>> np.allclose(sp_corr, th_corr)
    """
    mean_x = torch.mean(x)
    mean_y = torch.mean(y)

    xm = x.sub(mean_x)
    ym = y.sub(mean_y)
    
    
    r_den = torch.norm(xm, 2) * torch.norm(ym, 2)
    r_num = xm.dot(ym)
    
    r_val = r_num / r_den
    return r_val

In [74]:
import numpy as np
from scipy import linalg


def scipy_pearsonr_np(x,y):

    n = len(x)
    if n != len(y):
        raise ValueError('x and y must have the same length.')

    if n < 2:
        raise ValueError('x and y must have length at least 2.')


    if (x == x[0]).all() or (y == y[0]).all():
        msg = ("An input array is constant; the correlation coefficient "
                "is not defined.")
        
    xmean = x.mean()
    ymean = y.mean()

    xm = x - xmean
    ym = y - ymean

    # np.linalg.norm or sqrt((xm*xm).sum()),
    normxm = linalg.norm(xm)
    normym = linalg.norm(ym)
        
    r = np.dot(xm/normxm, ym/normym)

    # Presumably, if abs(r) > 1, then it is only some small artifact of
    # floating point arithmetic.
    r = max(min(r, 1.0), -1.0)

    return r

In [75]:
import torch
def pearsonr_torch(x,y):
    '''torch version of scipy.pearsonr'''

    n = len(x)
    if n != len(y):
        raise ValueError('x and y must have the same length.')

    if n < 2:
        raise ValueError('x and y must have length at least 2.')


    if (x == x[0]).all() or (y == y[0]).all():
        print("An input array is constant; the correlation coefficient "
                "is not defined.")
        return np.nan

    xmean = torch.mean(x)
    ymean = torch.mean(y)

    xm = torch.sub(x, xmean)
    ym = torch.sub(y, ymean)

    normxm = torch.linalg.norm(xm)
    normym = torch.linalg.norm(ym)
    
    r = torch.matmul(xm/normxm, ym/normym)
    
    # Presumably, if abs(r) > 1, then it is only some small artifact of
    # floating point arithmetic.
    r = max(min(r, 1.0), -1.0)
    
    return r

In [90]:
x = np.random.rand(100,)
y = np.random.rand(100,)

# check with the original function
from scipy.stats import pearsonr
r, p = pearsonr(x, y)
print(r)

r = scipy_pearsonr_np(x,y)
print(r)

-0.03279699949421083
-0.03279699949421083


In [91]:
torch.set_printoptions(precision=18)

x = torch.tensor(x)
y = torch.tensor(y)

r = pearsonr_torch(x,y)
print(r.item())

-0.03279699949421086
